## Preparation

In [1]:
# Use bash in .cms cluster
export workdir=`pwd`
cd $workdir
rm -rf daq
svn export https://svn.cern.ch/reps/cmsos/branches/bril/daq daq
    

/home/zhen/work/pynb/sandbox/zx/notebooks


## Set up working environment

In [5]:
# Go to scripts directory where all the controls are
%cd $workdir/daq/bril/scripts

/home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts


In [6]:
# Look at scripts directory contents
%ls

brilenv.sh*          generatexml.sh*  ReadMe.txt             timesource/
default.profile      generator.txt    spec.template          webprogramming.txt
eventing/            lumistore/       stopApp.sh*            xaas_bril.txt
freeportscanner.py*  Makefile         test_brilcondacore.py


In [7]:
# scripts/brilenv.sh sets runtime bash environment
# It must be run from inside the scripts directory
# It must be run in every new shell
! source ./brilenv.sh

LD_LIBRARY_PATH is  /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/../../../x86_64_slc6/lib:/opt/brilconda/lib:/opt/xdaq/lib:/home/zhen/brilconda/lib:/home/zhen/brilconda/root/lib
PATH is  /opt/brilconda/bin:/home/zhen/brilconda/bin:/home/zhen/brilconda/root/bin:/usr/lib/jvm/java-1.6.0/bin:/home/zhen/scripts:/home/zhen/brilconda/bin:/home/zhen/brilconda/root/bin:/usr/lib/jvm/java-1.6.0/bin:/usr/sue/bin:/opt/WinCC_OA/3.11/bin:/usr/lib64/qt-3.3/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/sbin:/usr/sbin:/sbin:/home/zhen/bin:/home/zhen/bin


## Compile Application Package (skip if done once today)

In [8]:
# Change to the package directory in order to compile
%cd $workdir/daq/bril/timesource

/home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/timesource


In [9]:
%%bash --out output --err error
make install

## Application configuration templates and their parameter file

<p> 
   <pre>
    scripts/$package/myapp.template  this is the xdaq configuration file, except variables can be templated.
    scripts/$package/template.param  template.param file contains substitutions of parameters defined in all the templates 
                                     (you might have more than one).
    </pre>
</p>
* in the template file, "%" enclosed are template variables.
* "%brildaqroot%" is special, when empty, it can be deduced
* The rightside value is bash expanded. For example, the formats below are allowed and evaluated.
    <pre>
       %a%=`ls` 
       %b%=$BVAL 
   </pre>



In [10]:
# Go to scripts directory to work with configurations
%cd $workdir/daq/bril/scripts

/home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts


In [11]:
# Look at the current application config template 
%cat $workdir/daq/bril/scripts/timesource/timesource_datagenerator.template

<?xml version="1.0" encoding="UTF-8"?>
<xc:Partition xmlns:soapenc="http://schemas.xmlsoap.org/soap/encoding/" xmlns:xc="http://xdaq.web.cern.ch/xdaq/xsd/2004/XMLConfiguration-30" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">

 <xc:Context url="http://%hostname%:%tscontextport%">
    <xc:Endpoint protocol="utcp" service="b2in" rmode="select" hostname="%hostname%" port="%tsendpoint%" network="utcp1" sndTimeout="0" rcvTimeout="0" affinity="RCV:P,SND:W,DSR:W,DSS:W" singleThread="true" publish="true"/>

    <xc:Application class="pt::utcp::Application" id="12" instance="0" network="local">
      <properties xmlns="urn:xdaq-application:pt::utcp::Application" xsi:type="soapenc:Struct">
	<maxClients xsi:type="xsd:unsignedInt">10</maxClients>
        <autoConnect xsi:type="xsd:boolean">false</autoConnect>
        <ioQueueSize xsi:type="xsd:unsignedInt">65536</ioQueueSize>
        <eventQueueSize xsi:type="xsd:unsignedInt">65536</eventQueueSize>
        <maxReceiveBuffers xsi:type="xsd

In [12]:
# Look at the current template.param
# Note: all the template variables in the previous file are defined here
# Note: localdaqroot is special, when the righthand side is whitespace, it means the generator script will guess it
%cat $workdir/daq/bril/scripts/timesource/template.param

%hostname%=`hostname -f`
%tscontextport%=40000
%tsendpoint%=400001
%dgcontextport%=40001
%dgendpoint%=400011
%eventinghostname%=`hostname -f`
%utcpport%=30020
%localdaqroot%=
#EOF need this fake lastline if using ipython notebook editor#


In [13]:
# Generate application configuration files based on templates and parameters in directory timesource
!./generatexml.sh timesource

sources:
    timesource/template.param,timesource/timesource_datagenerator.template
generated:
    /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator.xml
command per context:
   /opt/xdaq/bin/xdaq.exe -p 40000 -c /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator.xml 
   /opt/xdaq/bin/xdaq.exe -p 40001 -c /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator.xml 

sources:
    timesource/template.param,timesource/timesource_datagenerator_inzone.template
generated:
    /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator_inzone.xml
command per context:
   /opt/xdaq/bin/xdaq.exe -p 40000 -c /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator_inzone.xml -z bril
   /opt/xdaq/bin/xdaq.exe -p 40001 -c /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator_inzone.xml -z bril



<div style="color:DarkCyan">
    The output of this script suggests possible xdaq commands to run. <br>
    In this example, it configures two xdaq executives. <br>
    The first Context block is the one we are going to run. This is a timing simulator and a b2in eventing publisher. <br>
    Since this notebook server is not in bril zone, we are going to run the configuratio without _inzone.<br>
    Ready to go?
    Wait...all the same xdaq applications running on this notebook server, port 40000 will not be free for everyone.<br>
    We need to edit the port numbers to free ones.
</div>

## Configure XDAQ Application

In [14]:
# Scan for 10 free ports in default range [30000,30010)
%run freeportscanner.py 
# Scan for 10 free ports starting from the mininal value
%run freeportscanner.py 50000 
# Scan for free ports in a precise range
%run freeportscanner.py 50000 50020

scan range: [30000,30010)
free ports: [30000, 30002, 30003, 30004, 30005, 30006, 30007, 30008, 30009]
scan range: [50000,50010)
free ports: [50000, 50001, 50002, 50003, 50004, 50005, 50006, 50007, 50008, 50009]
scan range: [50000,50020)
free ports: [50000, 50001, 50002, 50003, 50004, 50005, 50006, 50007, 50008, 50009, 50010, 50011, 50012, 50013, 50014, 50015, 50016, 50017, 50018, 50019]


In [15]:
# Select your context port number and endpoint port number, and remember it
# This Application is a b2in publisher, it uses two ports: http for xdaq context and endpoint for pt transport
contextport = 50002
endpointport = 50003

<div style="color:DarkCyan">
Open the template file <br>
modify %tscontextport% and %tsendpoint% <br>
Execution of the command will overwrite file timesource/template.param
</div>

In [16]:
%cat $workdir/daq/bril/scripts/timesource/template.param

%hostname%=`hostname -f`
%tscontextport%=40000
%tsendpoint%=400001
%dgcontextport%=40001
%dgendpoint%=400011
%eventinghostname%=`hostname -f`
%utcpport%=30020
%localdaqroot%=
#EOF need this fake lastline if using ipython notebook editor#


<div style="color:DarkCyan">
 Replace the line "To Paste" with the output of last command, then modify tscontextport, tsendpoint values. Control-Enter to save file
</div>

In [17]:
%%file $workdir/daq/bril/scripts/timesource/template.param
%hostname%=`hostname -f`
%tscontextport%=50002
%tsendpoint%=50003
%dgcontextport%=40001
%dgendpoint%=400011
%eventinghostname%=`hostname -f`
%utcpport%=30020
%localdaqroot%=
#EOF need this fake lastline if using ipython notebook editor#


Overwriting /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource/template.param


In [18]:
# Generate application configuration again to get your runnable version
!./generatexml.sh timesource

sources:
    timesource/template.param,timesource/timesource_datagenerator.template
generated:
    /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator.xml
command per context:
   /opt/xdaq/bin/xdaq.exe -p 50002 -c /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator.xml 
   /opt/xdaq/bin/xdaq.exe -p 40001 -c /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator.xml 

sources:
    timesource/template.param,timesource/timesource_datagenerator_inzone.template
generated:
    /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator_inzone.xml
command per context:
   /opt/xdaq/bin/xdaq.exe -p 50002 -c /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator_inzone.xml -z bril
   /opt/xdaq/bin/xdaq.exe -p 40001 -c /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator_inzone.xml -z bril



In [19]:
# Look at the configuration you want to run, simbol % should not be there if template substitution worked
%cat ./timesource_datagenerator.xml

<?xml version="1.0" encoding="UTF-8"?>
<xc:Partition xmlns:soapenc="http://schemas.xmlsoap.org/soap/encoding/" xmlns:xc="http://xdaq.web.cern.ch/xdaq/xsd/2004/XMLConfiguration-30" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">

 <xc:Context url="http://pczhen.cern.ch:50002">
    <xc:Endpoint protocol="utcp" service="b2in" rmode="select" hostname="pczhen.cern.ch" port="50003" network="utcp1" sndTimeout="0" rcvTimeout="0" affinity="RCV:P,SND:W,DSR:W,DSS:W" singleThread="true" publish="true"/>

    <xc:Application class="pt::utcp::Application" id="12" instance="0" network="local">
      <properties xmlns="urn:xdaq-application:pt::utcp::Application" xsi:type="soapenc:Struct">
	<maxClients xsi:type="xsd:unsignedInt">10</maxClients>
        <autoConnect xsi:type="xsd:boolean">false</autoConnect>
        <ioQueueSize xsi:type="xsd:unsignedInt">65536</ioQueueSize>
        <eventQueueSize xsi:type="xsd:unsignedInt">65536</eventQueueSize>
        <maxReceiveBuffers xsi:type="xsd:unsigned

## Run XDAQ application and check its web page

<div style="color:DarkCyan">
    Pick the command to run from generatexml.sh output.
    Copy and paste it inside the quote of xdaqcommand
</div>

In [20]:
#
# Pick and Copy-Paste the command to run in the double quotes of xdaqcmd .
# 'source brilenv.sh' is run again because we are going to send xdaq to a python subprocess to run in the background.
# And we want that new shell has the correct environment.
# If in a normal main bash session, you don't need to source again
#
xdaqcmd="/opt/xdaq/bin/xdaq.exe -p 50002 -c /home/zhen/work/pynb/sandbox/zx/notebooks/daq/bril/scripts/timesource_datagenerator.xml"
mycmd="source ./brilenv.sh; "+xdaqcmd

## Extra about .cms environment

<div style="color:Darkgreen">
This IPython notebook virtual environment is identical to brildev installations in .cms network except that it does not simulate inside .cms firewall setup.

Working with real online nodes, if you want to see inside .cms applications through your outside browser, you need to setup proxy and ssh tunnel

The following setting need to be done once (on the browser you use to see .cms)

in browser url field type "about:config"
 <pre>
->search "proxy" 
   make sure the following values
  ->network.proxy.type value 1
  ->network.proxy.socks value localhost
  ->network.proxy.socks_port value 10880
  ->network.proxy.socks_remote_dns value true
  ->network.proxy.no_proxies_on add your noproxy whitelist,e.g. localhost,127.0.0.1,.fr,.ch,.us,.gov,.edu
    This list can be also modified anytime later from browser(firefox) Preference -> Advanced-> Network->Settings menu    
  </pre>
  
  
  ssh tunnel :
  <pre>
    ssh -D 10880 yourusername@cmsusr
    password
    </pre>
  open browser and test url:
  <pre>
     srv-c2d05-18.cms:30001
     </pre>
 </div>    